# Context-Aware Recommendation Algorithm by PAPERDAA-2

In [34]:
    import numpy as np
    import pandas as pd
    import math
    from sklearn import preprocessing
    from itertools import product as cartesian_product

# Training

## Program Arguments

In [35]:
LMAX = 10    # Maximum transitive path length
K = 50       # Nearest neighbors

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


## Preprocessing

In [36]:
df = pd.read_csv('training_dataset.csv')

df.tail(5)

,userid,itemid,rating,Time,Location,Companion
3280,1082,tt0413267,1,Weekend,Home,Partner
3281,1082,tt1637706,2,Weekend,Home,Partner
3282,1082,tt0343660,1,Weekend,Home,Partner
3283,1082,tt1133985,1,Weekend,Home,Partner
3284,1082,tt1099212,1,Weekend,Home,Partner


In [37]:
encoder = preprocessing.LabelEncoder()

user_item_context_encodings = []
user_item_context_reverse_encodings = []
maximum_rating = df[df.columns[2]].max()

encoded_df = df.copy()
# Encode userid, itemid, and contextual informations for item splitting
for column_index in range(len(df.columns)):
    
    # Column attribute is not rating
    if column_index != 2:
        
        # Fit encoder
        encoder.fit(df[df.columns[column_index]])
        encoded_df[df.columns[column_index]] = encoder.transform(
                df[df.columns[column_index]]
            )
    
    # Column is nor user or rating
    if column_index != 2:
            user_item_context_encodings.append(
                dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
            )
            user_item_context_reverse_encodings.append(
                dict(zip(encoder.transform(encoder.classes_), encoder.classes_))
            )

In [38]:
display(encoded_df)

,userid,itemid,rating,Time,Location,Companion
0,2,38,1,0,0,0
1,2,8,1,0,0,0
2,2,74,1,0,0,0
3,2,5,1,0,0,0
4,2,2,1,0,0,0
...,...,...,...,...,...,...
3280,65,35,1,1,1,2
3281,65,62,2,1,1,2
3282,65,25,1,1,1,2
3283,65,50,1,1,1,2


## Item Splitting

In [39]:
# Cartesian product all items and contexts

users = user_item_context_encodings[0].values()
items = user_item_context_encodings[1].values()
contexts = [
    context_trans.values() for context_trans in user_item_context_encodings[2:]
]

context_T = list(cartesian_product(items, *contexts))

In [40]:
# Generate new user-item matrix for new items
rating_matrix = np.zeros((len(users), len(context_T)), dtype=object)

In [41]:
for row in encoded_df.iterrows():
    data = tuple(row[1])
    user = data[0]
    item = data[1]
    rating = data[2]
    context_item = (item, *data[3:])
    
    index = context_T.index(context_item)
    
    rating_matrix[user][index] = int(rating)/int(maximum_rating)

In [42]:
display(rating_matrix)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0.2, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1.0, 0.8, 1.0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

## Graph Similarity Calculation (User-Based)

In [43]:
rating_matrix_transposed = np.transpose(rating_matrix)

In [44]:
L = 2

WWT = np.matmul(rating_matrix, rating_matrix_transposed)
M = np.matmul(rating_matrix, rating_matrix_transposed)

while L != LMAX:
    M = np.matmul(WWT, M)
    L = L + 2

In [45]:
display(M)

array([[143158955.47527495, 102667981.8717269, 34036220.62064927, ...,
        240328916.70763066, 360026156.222656, 114083506.52390018],
       [102667981.87172689, 98728489.39962108, 28269786.896847673, ...,
        182059944.62670246, 260235166.81354448, 88468759.8770003],
       [34036220.62064927, 28269786.896847673, 9369766.55159245, ...,
        55505184.19374091, 92779145.02201876, 28275329.82950585],
       ...,
       [240328916.70763075, 182059944.6267024, 55505184.19374091, ...,
        435336920.95773375, 582368224.3299332, 195102508.41326606],
       [360026156.2226561, 260235166.81354442, 92779145.02201878, ...,
        582368224.3299332, 1045657386.5458099, 297043639.9541597],
       [114083506.52390018, 88468759.8770003, 28275329.829505846, ...,
        195102508.41326615, 297043639.95415974, 97219155.64774357]],
      dtype=object)

# Prediction

In [46]:
USER = 1058   # User to check
N = 10

## Ratings Prediction

In [47]:
# Get K most similar users
wanted_user = user_item_context_encodings[0][USER]

scores = M[wanted_user]
K_similar_users = np.argpartition(scores, -(K+1))[-(K+1):]

if wanted_user in K_similar_users:
    K_similar_users = K_similar_users[K_similar_users != wanted_user]
else:
    K_similar_users = K_similar_users[:-1]

### Get list of rated items

In [48]:
inferred_ratings = []
user_rated_items = rating_matrix[wanted_user]

### KNN

In [49]:
for item in range(len(user_rated_items)):
    rating = user_rated_items[item]
    
    # Item has not been rated
    if rating == 0:
        rating_sum = 0
        neighbor_count = 0
        
        for neighbor in K_similar_users:
            neighbor_rating = rating_matrix[neighbor][item]
            
            # If neighbor has rated the item
            if neighbor_rating != 0:
                rating_sum += neighbor_rating
                neighbor_count += 1
            
        inferred_rating = rating_sum / neighbor_count if neighbor_count != 0 else 0
        
        if inferred_rating != 0:
            inferred_ratings.append((item, inferred_rating))

In [50]:
mapped_ratings = []
for i in inferred_ratings:
    
    # Get actual item encoding (from the cartesian product result)
    item_context = context_T[i[0]]
    
    item = user_item_context_reverse_encodings[1][item_context[0]]
    
    new_entry = [item]
    
    for j in range(1, len(item_context)):
        new_entry.append(item_context[j])
    
    new_entry.append(i[1])
    
    mapped_ratings.append(new_entry)

In [51]:
predicted_df = pd.DataFrame(
    mapped_ratings, 
    columns=[
        'Item', 
        *encoded_df.columns[3:], 
        'predicted_rating']
).sort_values(by='predicted_rating', ascending=False)
display(predicted_df)

,Item,Time,Location,Companion,predicted_rating
162,tt0232500,0,0,2,1.0
338,tt0441773,1,1,2,1.0
111,tt0138097,1,1,0,1.0
619,tt2096673,1,0,1,1.0
707,ttnanana2,0,0,2,1.0
...,...,...,...,...,...
255,tt0362165,0,1,2,0.2
254,tt0362165,0,1,0,0.2
686,tt4052394,1,1,2,0.2
48,tt0114148,0,0,2,0.2


## Recommendation Generation

In [52]:
# Select N items to be recommended
chosen = []

for data in predicted_df.iterrows():
    data_tup = tuple(data[1])
      
    if len(chosen) <= N:
        chosen.append(data_tup)

for i in chosen:
    print(i)

('tt0232500', 0, 0, 2, 1.0)
('tt0441773', 1, 1, 2, 1.0)
('tt0138097', 1, 1, 0, 1.0)
('tt2096673', 1, 0, 1, 1.0)
('ttnanana2', 0, 0, 2, 1.0)
('tt0315733', 1, 0, 0, 1.0)
('tt0376541', 0, 0, 0, 1.0)
('tt0110475', 0, 1, 1, 1.0)
('tt0407304', 0, 0, 0, 1.0)
('tt0405422', 1, 1, 2, 1.0)
('tt0376541', 0, 0, 2, 1.0)


## Rating Prediction Translated Result

In [53]:
mapped_ratings = []
for i in inferred_ratings:
    
    # Get actual item encoding (from the cartesian product result)
    item_context = context_T[i[0]]
    
    item = user_item_context_reverse_encodings[1][item_context[0]]
    
    new_entry = [item]
    
    # Translate remaining context
    for j in range(1, len(item_context)):
        # 1 --> item
        # >= 2 --> context
        translator = user_item_context_reverse_encodings[j+1]
        new_entry.append(translator[item_context[j]])
    
    # Append the rating
    new_entry.append(i[1]*maximum_rating)
    
    mapped_ratings.append(new_entry)

In [54]:
res = pd.DataFrame(
    mapped_ratings, 
    columns=[
        'Item', 
        *encoded_df.columns[3:], 
        'predicted_rating']
).sort_values(by='predicted_rating', ascending=False)
display(res)

,Item,Time,Location,Companion,predicted_rating
162,tt0232500,Weekday,Cinema,Partner,5.0
338,tt0441773,Weekend,Home,Partner,5.0
111,tt0138097,Weekend,Home,Alone,5.0
619,tt2096673,Weekend,Cinema,Family,5.0
707,ttnanana2,Weekday,Cinema,Partner,5.0
...,...,...,...,...,...
255,tt0362165,Weekday,Home,Partner,1.0
254,tt0362165,Weekday,Home,Alone,1.0
686,tt4052394,Weekend,Home,Partner,1.0
48,tt0114148,Weekday,Cinema,Partner,1.0


In [55]:
res[res['Item'] == 'tt0266543']

,Item,Time,Location,Companion,predicted_rating
175,tt0266543,Weekday,Home,Partner,5.000000
176,tt0266543,Weekend,Cinema,Family,4.666667
173,tt0266543,Weekday,Cinema,Partner,4.333333
178,tt0266543,Weekend,Home,Family,4.111111
172,tt0266543,Weekday,Cinema,Family,4.100000
177,tt0266543,Weekend,Home,Alone,4.000000
174,tt0266543,Weekday,Home,Alone,3.833333
179,tt0266543,Weekend,Home,Partner,3.500000
171,tt0266543,Weekday,Cinema,Alone,3.000000


## Dataset Merge

In [56]:
witheld = pd.read_csv('witheld_ratings.csv')

In [57]:
display(witheld)

,userid,itemid,rating,Time,Location,Companion
0,1067,tt0378194,2,Weekday,Cinema,Partner
1,1067,tt0111161,1,Weekday,Cinema,Partner
2,1067,tt1632708,1,Weekday,Cinema,Partner
3,1067,tt0356910,4,Weekend,Cinema,Partner
4,1067,tt0378194,1,Weekend,Cinema,Partner
...,...,...,...,...,...,...
147,1058,tt0319262,3,Weekday,Home,Family
148,1058,tt1657301,3,Weekday,Home,Family
149,1058,ttnanana1,2,Weekday,Cinema,Partner
150,1058,tt3793764,3,Weekday,Home,Family


In [58]:
actual_ratings = []

In [59]:
for row in res.iterrows():
    row_data = row[1]
    actual = witheld[
        (witheld['userid'] == USER) &\
        (witheld['itemid'] == row_data['Item']) &\
        (witheld['Time'] == row_data['Time']) &\
        (witheld['Location'] == row_data['Location']) &
        (witheld['Companion'] == row_data['Companion'])
    ]['rating']
    
    if(not actual.empty):
        actual_ratings.append(tuple(actual)[0])
        
    else:
        actual_ratings.append(np.nan)   

In [60]:
merged_result = res.copy()

In [61]:
merged_result['actual_rating'] = actual_ratings

In [62]:
merged_result['actual_rating']

162   NaN
338   NaN
111   NaN
619   NaN
707   NaN
       ..
255   NaN
254   NaN
686   NaN
48    NaN
448   NaN
Name: actual_rating, Length: 715, dtype: float64

In [63]:
merged_result[
    (merged_result['actual_rating'] != np.nan)
]

,Item,Time,Location,Companion,predicted_rating,actual_rating
162,tt0232500,Weekday,Cinema,Partner,5.0,NaN
338,tt0441773,Weekend,Home,Partner,5.0,NaN
111,tt0138097,Weekend,Home,Alone,5.0,NaN
619,tt2096673,Weekend,Cinema,Family,5.0,NaN
707,ttnanana2,Weekday,Cinema,Partner,5.0,NaN
...,...,...,...,...,...,...
255,tt0362165,Weekday,Home,Partner,1.0,NaN
254,tt0362165,Weekday,Home,Alone,1.0,NaN
686,tt4052394,Weekend,Home,Partner,1.0,NaN
48,tt0114148,Weekday,Cinema,Partner,1.0,NaN


In [64]:
compared_df = merged_result.dropna()
display(compared_df)

,Item,Time,Location,Companion,predicted_rating,actual_rating
287,tt0382625,Weekday,Home,Family,4.000000,4.0
126,tt0169547,Weekend,Cinema,Alone,4.000000,5.0
545,tt1478338,Weekend,Cinema,Partner,4.000000,3.0
4,tt0088763,Weekday,Home,Family,4.000000,5.0
136,tt0181689,Weekend,Cinema,Alone,4.000000,5.0
672,tt3793764,Weekday,Home,Family,4.000000,3.0
119,tt0147800,Weekend,Cinema,Partner,4.000000,3.0
26,tt0110357,Weekend,Cinema,Partner,4.000000,3.0
219,tt0319262,Weekday,Home,Family,3.750000,3.0
264,tt0367594,Weekend,Cinema,Alone,3.500000,3.0


In [65]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
print("MAE", mean_absolute_error(
    compared_df['actual_rating'],
    compared_df['predicted_rating']
))

print("RMSE", mean_squared_error(
    compared_df['actual_rating'],
    compared_df['predicted_rating']
))

MAE 1.08030303030303
RMSE 1.7770959595959595
